In [2]:
!pip install scikit-optimize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 3.8 MB/s eta 0:00:00


In [6]:
""" ******************************************************************************** """
import time
start = time.time()
print("The starttime is :", (start) * 10**3/60, " Seconds")
""" ******************************************************************************** """
import cv2
import os, sys
import shap
import pandas            as pd
import tensorflow        as tf
import numpy             as np
import matplotlib.pyplot as plt
import seaborn           as sns
from sklearn                       import metrics
from sklearn.metrics               import accuracy_score, confusion_matrix,  classification_report, f1_score, roc_curve, roc_auc_score, ConfusionMatrixDisplay
from sklearn.model_selection       import train_test_split
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.utils        import to_categorical
from keras.callbacks               import EarlyStopping
from tensorflow.keras.models       import Sequential, Model
from tensorflow.keras.layers       import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.applications import VGG16, ResNet50, MobileNetV2
from tensorflow.keras.applications import InceptionResNetV2, InceptionV3
from tensorflow.keras.applications import DenseNet201, Xception, EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input

from sklearn.svm             import SVC
from sklearn.preprocessing   import StandardScaler
from sklearn.ensemble        import RandomForestClassifier
from sklearn.tree            import DecisionTreeClassifier,  plot_tree
from scipy.stats             import randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.datasets        import make_classification
from sklearn.model_selection import KFold,cross_val_score
from sklearn.model_selection import GridSearchCV
from skopt                   import BayesSearchCV #pip install scikit-optimize
from sklearn.neighbors       import KNeighborsClassifier

The starttime is : 29393639014.87099  Seconds


In [8]:
class Configurations:

  #All these parameter should be changed based on your dataset
  Data_Path       = "E:\\CMT Mammary Tumors\\Datasets\\CMT Dataset\\"

  model_path   = "Best_Model_Weights.h5"
  CMT_labels   = ['benign', 'malignant']
  CMT_values   = [0       ,     1      ]
  Num_Classes  = len(CMT_labels)
  CNN_Models   = ['Basic', 'VGG', 'ResNet50','MobileNet','InceptionResNet', 'Inception', 'Xception', 'EfficientNet', 'DenseNet201']
  augments     = [True , False]

  IMAGE_WIDTH  = 224 #128 224 256
  IMAGE_HEIGHT = 224 #128 224 256
  test_size    = 0.20
  learning_rate= 0.001
  batch_size   = 64
  val_size     = 0.5 #50% of test as validate%
  epochs       = 1
  Current_CNN  = 'DenseNet201'

  classifiers  = ['softmax', 'random_forest', 'svc', 'decision_tree', 'knn']
  classifier   = 'knn'
  augment      = False
  patience     = 2

  split_methods= ['splitNsamples', 'splittwice', 'splitfolders']
  split_method = 'splittwice'

  optimizeParam= False
  optimizers   = ['RandomizedSearchCV' , 'GridSearchCV', 'BayesSearchCV']
  optimizer    = 'RandomizedSearchCV'

  def test_gpu(self):

      print ("Python version is     : " , sys.version   )
      print ("tensorflow version is : " , tf.__version__)

      # Get detailed information about GPU(s)
      gpus = tf.config.experimental.list_physical_devices('GPU')
      if gpus:
          print("\nTensorFlow is using the following GPU(s):")
          for gpu in gpus:
              print(gpu)
      else:
          print("\nNo GPU detected or TensorFlow is not configured to use one.")


In [9]:
class CNN:

  def One_Hot_Encoding(self, y_train, num_classes ):
    y_train = to_categorical(y_train , num_classes=num_classes)
    return y_train

  def model_save(self, model , model_path):
    model.save(model_path)

  def plot_history(self, history):#call after model.fit

      plt.figure(figsize=(12, 5))

      # Plot loss
      plt.subplot(1, 2, 1)
      plt.plot(history.history['loss']    , label='train_loss')
      plt.plot(history.history['val_loss'], label='val_loss')
      plt.legend()
      plt.xlabel('Epochs')
      plt.ylabel('Loss')
      plt.title('Loss over Epochs')

      # Plot accuracy
      plt.subplot(1, 2, 2)
      plt.plot(history.history['accuracy']    , label='train_accuracy')
      plt.plot(history.history['val_accuracy'], label='val_accuracy')
      plt.legend()
      plt.xlabel('Epochs')
      plt.ylabel('Accuracy')
      plt.title('Accuracy over Epochs')
      plt.legend(loc='lower right')

      plt.tight_layout()

      # Save the figure
      plt.savefig('Loss_accuracy.png', dpi=300, bbox_inches='tight')
      #plt.show()

  def plot_confusion_matrix(self, predictions, y_test):

      y_pred                 = np.argmax(predictions, axis=1)  # Get the index of the maximum value
      y_true                 = np.argmax(y_test, axis=1)
      cm                     = confusion_matrix(y_true, y_pred)

      cmd                    = ConfusionMatrixDisplay(confusion_matrix=cm)
      cmd.plot(include_values= True, cmap='viridis', ax=None, xticks_rotation='horizontal')

      # Save the figure
      plt.savefig('Confusion Matrix.png', dpi=300, bbox_inches='tight')
      #plt.show()

  def plot_classification_report(self,  predictions, y_test , CMT_labels, Num_Classes):

      predicted_classes = [np.argmax(pred) for pred in predictions]

      classes           = CMT_labels
      predicted_classes = self.One_Hot_Encoding(predicted_classes, Num_Classes)

      print(classification_report(y_test, predicted_classes))

      report            = classification_report(y_test, predicted_classes, target_names=classes, output_dict=True)
      metrics           = {label: report[label]        for label in classes if label in report}
      precision         = [metrics[label]['precision'] for label in classes]
      recall            = [metrics[label]['recall']    for label in classes]
      f1_score          = [metrics[label]['f1-score']  for label in classes]

      data = {
              'Precision': precision,
              'Recall'   : recall,
              'F1-Score' : f1_score
              }

      df   = pd.DataFrame(data, index=classes)

      plt.figure(figsize=(10, 6))
      sns.heatmap(df, annot=True, cmap='Blues', fmt=".2f", linewidths=0.5)
      plt.title('Classification Report')
      plt.xlabel('Metrics')
      plt.ylabel('Classes')

      # Save the figure
      plt.savefig('Classification Report.png', dpi=300, bbox_inches='tight')

      #plt.show()

  def plot_predictions(self, predictions, x_test, y_test):

    sample_classes = np.argmax(predictions, axis = 1)
    sizeofplot     = 3 #subplot 3*3
    fig, ax        = plt.subplots(sizeofplot,sizeofplot, figsize=(5,5))
    index          = 0

    for i in range(sizeofplot):
        for j in range(sizeofplot):

          if y_test[index].argmax() == 0:#argmax since one hot
              true = "benign"
          else:
              true = "malignant"

          if sample_classes[index] == 0:
              pred = "benign"
          else:
              pred = "malignant"

          color   = "green"
          if true != pred:
             color="red"

          ax[i][j].imshow(tf.squeeze(x_test[index]), cmap=plt.cm.gray)
          ax[i][j].set_title("True: " + true + " Predicted: "+ pred, color = color )
          plt.axis("off")
          index = index + 1

    # Save the figure
    plt.savefig('Predictions.png', dpi=300, bbox_inches='tight')

    fig.tight_layout()
    #plt.show()
    #cv2.waitKey(50)

  def plot_auc(self, predictions, y_test ):

    # Ensure that the target labels Y_test are in a 2-dimensional format
    if len(y_test.shape) == 1:
        y_test = np.eye(len(np.unique(y_test)))[y_test.astype(int)]

    # Compute the ROC curve and AUC score for each class
    fpr     = dict()
    tpr     = dict()
    roc_auc = dict()

    for i in range(y_test.shape[1]):
        fpr[i], tpr[i], _ = roc_curve(y_test[:, i], predictions[:, i])
        roc_auc[i]        = roc_auc_score(y_test[:, i], predictions[:, i])

    # Plot the ROC curve for each class
    plt.figure()
    for i in range(y_test.shape[1]):
        plt.plot(fpr[i], tpr[i], label=f'Class {i} (AUC = {roc_auc[i]:.2f})')

    # Set the title and axis labels
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(loc='lower right')

    # Save the figure
    plt.savefig('ROC.png', dpi=300, bbox_inches='tight')

    # Show the plot
    #plt.show()

  def plot_auc_all(self, predictions, y_test):

    plt.figure(0).clf()

    pred  = np.random.rand(1000)
    label = np.random.randint(2, size=1000)

    fpr, tpr, thresh = metrics.roc_curve(label, pred)
    auc   = metrics.roc_auc_score(label, pred)
    plt.plot(fpr,tpr,label="data 1, auc="+str(auc))

    pred  = np.random.rand(1000)
    label = np.random.randint(2, size=1000)

    fpr, tpr, thresh = metrics.roc_curve(label, pred)
    auc   = metrics.roc_auc_score(label, pred)
    plt.plot(fpr,tpr,label="data 2, auc="+str(auc))

    plt.legend(loc=0)

  def show_test_samples(self, test_path, pred_list, image_size):

    index = 0
    for item in os.listdir(test_path):

      if index < 3 :
          full_path  = test_path + item
          img        = tf.keras.preprocessing.image.load_img(full_path, target_size=(image_size, image_size))
          img_tensor = tf.keras.preprocessing.image.img_to_array(img)
          img_tensor = np.expand_dims(img_tensor, axis=0)
          img_tensor/= 255.
          pred       = model.predict(img_tensor)
          pred       = np.argmax(pred, axis = 1)[0]#model.predict(img_tensor)
          pred_list.append([img, pred])

          index = index+1

    return pred_list

  def plot_confusion_matrix_generator(self, predictions, y_test):

    y_pred                 = np.argmax(predictions, axis=1)  # Get the index of the maximum value
    cm                     = confusion_matrix(y_test, y_pred)

    cmd                    = ConfusionMatrixDisplay(confusion_matrix=cm)
    cmd.plot(include_values= True, cmap='viridis', ax=None, xticks_rotation='horizontal')

    # Save the figure
    plt.savefig('Confusion Matrix.png', dpi=300, bbox_inches='tight')

    #plt.show()

  def evaluate_x_test(self, model, history , x_test, y_test, Num_Classes, CMT_labels, batch_size):

    #1- Evaluate the model on the test data using `evaluate`
    _, accuracy = model.evaluate(x_test, y_test, batch_size=batch_size)
    print()
    print('evaluation accuracy is**************************************************************:  %.3f' % (accuracy * 100.0))
    print()

    #2- Draw the results of accuray, and loss
    self.plot_history(history)

    #3- Make predictions softmax
    y_pred  = model.predict(x_test)

    #4- Plot the confusion matrix of this model for the test data
    self.plot_confusion_matrix( y_pred, y_test)

    #5- Plot the classification report of this model for the test data
    self.plot_classification_report(y_pred, y_test, CMT_labels, Num_Classes)

    #6- Show the predition results as images
    #self.plot_predictions(y_pred , x_test, y_test)

    #7- Plot AUC: area under the curve
    self.plot_auc(y_pred, y_test)
    #self.plot_auc_all( y_pred, y_test)

  def evaluate_generator(self, model , history , test_generator, y_test, Num_Classes, CMT_labels, batch_size, image_size, test_path=""):

    #1- Evaluate the model on the test data using `evaluate`
    _, accuracy = model.evaluate(test_generator, batch_size=batch_size)
    print()
    print('evaluation accuracy is**************************************************************:  %.3f' % (accuracy * 100.0))
    print()

    #2- Draw the results of accuray, and loss
    self.plot_history(history)

    #3-make the predictions of the model using the testing data
    y_pred = model.predict(test_generator)

    #4-Plot the confusion matrix
    self.plot_confusion_matrix_generator(y_pred, y_test)

    #5- Plot the classification report of this model for the test data
    y_true                 = self.One_Hot_Encoding(y_test, Num_Classes)  # Get the index of the maximum value
    self.plot_classification_report(y_pred, y_true, CMT_labels, Num_Classes)

    #6- Plot AUC: area under the curve
    self.plot_auc(y_pred, y_test)
    #self.plot_auc_all( y_pred, y_test)


In [10]:
class Models:

  def CNN_Model(self, IMAGE_WIDTH, IMAGE_HEIGHT, Num_classes):
    print ("******************************************** Basic CNN Model *********************************************")
    model = Sequential()

    # Input 0 of layer "conv2d" is incompatible with the layer: expected axis -1 of input shape to have value 1,
    # but received input with shape (None, 128, 128, 3)
    model.add(Conv2D(8, (3, 3), padding='same',activation='relu', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 3)))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(16, (3, 3), padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(32, (3, 3), padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))

    return model

  def VGG_Model(self, IMAGE_WIDTH, IMAGE_HEIGHT):
      print ("******************************************** VGG Model *********************************************")
      vgg       = VGG16(input_shape = (IMAGE_WIDTH, IMAGE_WIDTH , 3) , weights = 'imagenet', include_top = False)

      return vgg

  def ResNet_Model(self, IMAGE_WIDTH, IMAGE_HEIGHT):
      print ("******************************************** ResNet Model *********************************************")
      resnet    =  ResNet50(input_shape=(IMAGE_WIDTH,IMAGE_HEIGHT,3), weights = 'imagenet',include_top=False)
      return resnet

  def MobileNet(self, IMAGE_WIDTH, IMAGE_HEIGHT):
      MobileNet = MobileNetV2      (input_shape = (IMAGE_WIDTH , IMAGE_HEIGHT ,3), weights = 'imagenet', include_top = False)
      return MobileNet

  def Inception_Model(self, IMAGE_WIDTH, IMAGE_HEIGHT):
      print ("******************************************** Inception Model *********************************************")
      inception = InceptionV3(input_shape = (IMAGE_WIDTH , IMAGE_HEIGHT ,3), weights = 'imagenet', include_top = False )
      return inception

  def InceptionResNet_Model(self, IMAGE_WIDTH, IMAGE_HEIGHT):
      print ("******************************************** InceptionResNet Model *********************************************")
      inceptionResnet = InceptionResNetV2(input_shape = (IMAGE_WIDTH , IMAGE_HEIGHT ,3), weights = 'imagenet', include_top = False )
      return inceptionResnet

  def DenseNet_Model(self, IMAGE_WIDTH , IMAGE_HEIGHT ):
      print ("******************************************** DenseNet Model *********************************************")
      DenseNet = DenseNet201(input_shape=(IMAGE_WIDTH , IMAGE_HEIGHT ,3), weights='imagenet', include_top=False, pooling='avg')
      return DenseNet

  def Exception_model(self, IMAGE_WIDTH, IMAGE_HEIGHT):
      print ("******************************************** Exception Model *********************************************")
      Exception = Xception(input_shape = (IMAGE_WIDTH , IMAGE_HEIGHT ,3), weights = 'imagenet', include_top = False )
      return Exception

  def EfficientNet_Model(self, IMAGE_WIDTH, IMAGE_HEIGHT):
      print ("******************************************** EfficientNet Model *********************************************")
      EfficientNet = EfficientNetB0(input_shape = (IMAGE_WIDTH , IMAGE_HEIGHT ,3), weights = 'imagenet', include_top = False )
      return EfficientNet

  def create_softmax_classifier(self, model, num_classes):

      for layer in model.layers:
            layer.trainable = False

      x     = Flatten()(model.output)
      x     = Dense(num_classes, activation = 'softmax')(x)
      model = Model(inputs = model.input, outputs = x)

      model.compile(optimizer= 'Adam',loss = 'categorical_crossentropy',metrics  = ['accuracy'])
      return model

  def create_svm_classifier(self, model, num_classes):

      for layer in model.layers:
            layer.trainable = False

      x     = Flatten()(model.output)
      x     = Dense(num_classes, activation='softmax', kernel_regularizer=l1_l2(l1=0.01, l2=0.01))(x)
      model = Model(inputs = model.input, outputs = x)

      model.compile(optimizer ='Adam', loss = 'squared_hinge', metrics  = ['accuracy'])
      #model.summary()
      return model

  def choose_Model(self, Current_CNN, IMAGE_WIDTH, IMAGE_HEIGHT, Num_Classes, classifier):

    if Current_CNN == 'Basic':
        model = self.CNN_Model(IMAGE_WIDTH, IMAGE_HEIGHT, Num_Classes)

    elif Current_CNN == 'VGG':
        model = self.VGG_Model(IMAGE_WIDTH, IMAGE_HEIGHT)

    elif Current_CNN == 'ResNet50':
        model = self.ResNet_Model(IMAGE_WIDTH, IMAGE_HEIGHT)

    elif Current_CNN == 'MobileNet':
        model = self.MobileNet(IMAGE_WIDTH, IMAGE_HEIGHT)

    elif Current_CNN == 'Inception':
        model = self.Inception_Model(IMAGE_WIDTH, IMAGE_HEIGHT)

    elif Current_CNN == 'InceptionResNet':
        model = self.InceptionResNet_Model(IMAGE_WIDTH, IMAGE_HEIGHT)

    elif Current_CNN == 'DenseNet201':
        model = self.DenseNet_Model(IMAGE_WIDTH, IMAGE_HEIGHT)

    elif Current_CNN == 'Xception':
        model = self.Exception_model(IMAGE_WIDTH, IMAGE_HEIGHT)

    elif Current_CNN == 'EfficientNet':
      model = self.EfficientNet_Model(IMAGE_WIDTH, IMAGE_HEIGHT)

    return model

  def choose_classifier(self,  x_train, y_train, x_test, y_test, classifier, IMAGE_WIDTH, IMAGE_HEIGHT, optimize, optimizer, CMT_labels, Num_Classes):

    model = Models.DenseNet_Model(IMAGE_WIDTH, IMAGE_HEIGHT)
    for layer in model.layers:
            layer.trainable = False

    train_features  = model.predict(x_train)
    train_features  = train_features.reshape(train_features.shape[0], -1)

    test_features   = model.predict(x_test)
    test_features   = test_features.reshape(test_features.shape[0], -1)

    if classifier == "svc":
        rf_model       = SVC()

    elif classifier == "decision_tree":
        rf_model       = DecisionTreeClassifier()

    elif classifier=="random_forest":

      if optimize:

        param_dist  = {
                        'n_estimators': [40, 50, 80, 100]
                      }

        if optimizer == 'RandomizedSearchCV':
          rf_model = RandomizedSearchCV(rf_model, param_dist)

        elif optimizer=='GridSearchCV':
          rf_model = GridSearchCV( rf_model,   param_dist)

        elif optimizer=='knn':
          rf_model = KNeighborsClassifier( rf_model,   param_dist)

        elif optimizer =='BayesSearchCV':
          rf_model = BayesSearchCV( rf_model, param_dist)

      else:
        rf_model       = RandomForestClassifier(n_estimators=50, random_state=42)

    rf_model.fit(train_features, y_train)
    y_pred         = rf_model.predict(test_features)

    print("accuracy  : ",    accuracy_score  (y_test, y_pred) )
    print ("f1-score : ",    f1_score        (y_test, y_pred  , average='macro'))
    cm                     = confusion_matrix(y_test, y_pred)
    cmd                    = ConfusionMatrixDisplay(confusion_matrix=cm)
    cmd.plot(include_values= True, cmap='viridis', ax=None, xticks_rotation='horizontal')
    plt.savefig('Confusion Matrix.png', dpi=300, bbox_inches='tight')
    #plt.show()

    self.plot_classification_report(y_pred, y_test , CMT_labels, Num_Classes)


In [11]:
class Data:

  def Load_preprocess_resize_scale(self, Data_Path, IMAGE_WIDTH, IMAGE_HEIGHT, ClassLabel1):

    # Download the training and validation data
    filenames  = os.listdir(Data_Path) #Contains two folder one for images of each class
    print("Folder Names = Class Labels: : " , filenames)

    Labels     = []
    data       = []

    for filename in filenames:

      images = os.listdir(Data_Path + filename)

      for image in images:
        image_path = Data_Path + "\\"+ filename +"\\"+image

        #Create Labels:
        Label  = filename
        if Label == ClassLabel1:
          Label = 0 #Put 0  for benign
        else:
          Label = 1 #Put 1  for malignant

        #Create Image
        image = cv2.imread(image_path)

        image = self.preprocess_sharpen(image)

        #Resize image into 128*128
        image = cv2.resize(image, (IMAGE_WIDTH , IMAGE_HEIGHT),interpolation = cv2.INTER_AREA)
        #image = resize(image, (IMAGE_WIDTH , IMAGE_HEIGHT,1))

        #print ("when using SHAP, Comment this line since it affects it working properly ... ")
        #Normalize image pixel values between 0 and 255.
        image = image.astype('float') / 255.0

        #Append both image, and labels in data array
        data.append([image, Label])

    print("Number of samples from both classes are: len (data)" , len(data))

    return data

  def preprocess_sharpen(self, image):

    kernel      = np.array([[0, -1, 0],
                            [-1, 5,-1],
                            [0, -1, 0]])

    image = cv2.filter2D(src=image, ddepth=-1, kernel=kernel)

    return image

  def plot_label_count(self, data, CMT_labels):

    #Plot count of label of each class in the dataset
    labels         = np.array([i[1] for i in data])
    elements_count = {}

    for element in labels:
      if element in elements_count:
          elements_count[element] += 1
      else:
          elements_count[element] = 1

    for key, value in elements_count.items():
      print(f"{key}: {value}")

    x = [CMT_labels[0]    , CMT_labels[1]]
    y = [elements_count[0], elements_count[1]]

    #sns.countplot(x='labels', data=y)

    for i in range(len(x)):
        plt.text(i, y[i]//2,y[i], ha = 'center')

    plt.bar(x, y, color = 'red')
    plt.title("The count of Each class in CMT dataset")
    plt.xlabel("CMT Classes")
    plt.ylabel("Count of Each Class")
    #plt.show()

  def plot_data_samples3(self, x_train, y_train , label_name, label_value, sizeofplot):

      fig, ax  = plt.subplots(sizeofplot,sizeofplot, figsize=(5,5))
      result   = np.where(y_train[0:30] == label_value)[0]
      index    = 0
      for i in range(sizeofplot):
        for j in range (sizeofplot):
            ax[i][j].imshow(tf.squeeze(x_train[result[index]]), cmap=plt.cm.gray)
            ax[i][j].set_title("Class : " + label_name)
            plt.axis("off")
            index =  index+1
      fig.tight_layout()
      #plt.show()

  def plot_samples_path(self, Data_Path, dataset_name, label_name):

    Data_Path = Data_Path + label_name+"\\"
    images  = os.listdir(Data_Path) #Contains images of each class

    index   = 0
    size_fig= 2
    fig, ax = plt.subplots(size_fig,size_fig, figsize=(5,5))
    for i in range(size_fig):
        for j in range (size_fig):
            image = Data_Path+images[index]
            image = cv2.imread(image)
            #image = self.preprocess_sharpen(image)
            ax[i][j].imshow(tf.squeeze(image), cmap=plt.cm.gray)
            ax[i][j].set_title("Class : " + label_name)
            plt.axis("off")
            index =  index+1
    #fig.tight_layout()
    plt.savefig(dataset_name+' '+label_name+' Samples.png', dpi=300, bbox_inches='tight')
    #plt.show()

  def plot_label_count_path(self):

    x = ["benign" , 'malignant', 'benign'    , 'malignant' ]
    y = [480, 576, 576, 576]

    #sns.countplot(x='labels', data=y)

    for i in range(len(x)):
        plt.text(i, y[i]//2,y[i], ha = 'center')

    plt.bar(x, y, color = 'red')
    plt.title("The count of Each class in CMT dataset")
    plt.xlabel("CMT Classes")
    plt.ylabel("Count of Each Class")
    plt.show()
    plt.savefig('Balanced Labels.png', dpi=300, bbox_inches='tight')

  def plot_counts_CMT_BreakHis(self):

    blue_bar  = (480, 576, 2480, 5429)
    orange_bar= (576, 576, 5429, 5429)

    # Numbers of pairs of bars you want
    N         = 4
    ind       = np.arange(N)
    width     = 0.3
    plt.figure(figsize=(10,5))
    plt.bar(ind,         blue_bar  , width, label='Unbalanced count')
    plt.bar(ind + width, orange_bar, width, label='Balanced count', color='r')
    plt.xlabel('CMT                                                                           BreakHis')
    plt.ylabel('Number of samples in each class')
    plt.title('Balancing number of samples in CMT and BreakHis Datasets')
    plt.xticks(ind + width / 2, ('Benign', 'Malignant', 'Benign', 'Malignant'))
    plt.legend(loc='best')
    #plt.show()
    #plt.savefig('Balanced Labels.png', dpi=300, bbox_inches='tight')
    plt.savefig('Balancing Label counts.png', dpi=300, bbox_inches='tight')

  def plot_all(self, Data_Path, dataset_name, x_train, y_train):

    image_path   = Data_Path
    dataset_name = "CMT"
    self.plot_samples_path(Data_Path, dataset_name, "benign")
    self.plot_samples_path(Data_Path, dataset_name, "malignant")
    self.plot_counts_CMT_BreakHis()

    #Plot samples of each class: benign and malignant
    self.plot_data_samples3(x_train, y_train , 'benign'   , 0, 3) #Benign hs label 0
    self.plot_data_samples3(x_train, y_train , 'malignant', 1, 3) #Malignant hs label 1

  def split_train_test(self, data, test_size, IMAGE_WIDTH, IMAGE_HEIGHT):

    X = np.array([i[0] for i in data]).reshape(-1, IMAGE_WIDTH, IMAGE_HEIGHT, 3)
    y = np.array([i[1] for i in data])

    x_train, x_test, y_train, y_test = train_test_split(X,y , test_size =test_size, shuffle = True, stratify=y, random_state=11)
    print ("x_train: ", x_train.shape)
    print ("y_train: ", y_train.shape)
    print ("x_test:  ", x_test.shape)
    print ("y_test:  ", y_test.shape)

    return x_train, y_train, x_test, y_test

  def split_test_val(self, x_test, y_test, val_size):

      nsamples_val= int(val_size*len(x_test))
      print ("nsamples_val: " , nsamples_val)

      x_val = x_test[-nsamples_val:]
      y_val = y_test[-nsamples_val:]

      x_test = x_test[:-nsamples_val]
      y_test = y_test[:-nsamples_val]

      print (len(x_val))
      print (len(y_val))
      print (len(x_test))
      print (len(x_test))

      return x_test, y_test, x_val, y_val

  def split_test_val_splittwice(self, x_test, y_test, val_size, IMAGE_WIDTH, IMAGE_HEIGHT):

    new_data = []
    for i in range(len(x_test)):
        image = x_test[i]
        label = y_test[i]
        new_data.append([image, label])

    X = np.array([i[0] for i in new_data]).reshape(-1, IMAGE_WIDTH, IMAGE_HEIGHT, 3)
    y = np.array([i[1] for i in new_data])

    x_test, x_val , y_test, y_val = train_test_split(X,y , test_size =val_size, shuffle = True, stratify=y, random_state=11)

    print ("x_val:  ", x_val.shape)
    print ("y_val:  ", y_val.shape)
    print ("x_test: ", x_test.shape)
    print ("y_test: ", y_test.shape)

    return x_test, y_test, x_val, y_val

  def Augment_x_train_flow(self, x_train, y_train, x_val, y_val, x_test, y_test,batch_size):

      datagen                       = ImageDataGenerator( rotation_range = 5,width_shift_range = 0.01, height_shift_range= 0.01)

      train_generator               = datagen.flow(x_train, y_train, batch_size = batch_size, seed= 42,save_prefix='',save_format='jpg')
      validate_generator            = datagen.flow(x_val  , y_val  , batch_size = batch_size, seed= 42,save_prefix='',save_format='jpg')

      test_gen                      = ImageDataGenerator(  ) #Image is already scaled and resize in load function
      test_generator                = test_gen.flow( x_test,  y_test )

      return train_generator, validate_generator, test_generator


In [ ]:

if __name__== '__main__':

    Config  = Configurations()
    Data    = Data()
    CNN     = CNN()
    Models  = Models()

    #Config.test_gpu()

    #Save images, and labels in array called data
    data                            = Data.Load_preprocess_resize_scale(Config.Data_Path,Config.IMAGE_WIDTH, Config.IMAGE_HEIGHT, Config.CMT_labels[0])

    #Split data into train and test with test_size ratio
    x_train, y_train, x_test, y_test= Data.split_train_test(data, Config.test_size, Config.IMAGE_WIDTH,  Config.IMAGE_HEIGHT)

    #Split test into val and test with test_size ratio
    #x_test, y_test, x_val, y_val      = Data.split_test_val( x_test, y_test, Config.val_size)
    x_test, y_test, x_val, y_val    = Data.split_test_val_splittwice( x_test, y_test, Config.val_size, Config.IMAGE_WIDTH, Config.IMAGE_HEIGHT)

    end1 = time.time()
    print("The duration of processing the data is :", (end1-start) /60, " Minutes")

    #Choose the model and create the CNN model
    model                           = Models.choose_Model(Config.Current_CNN, Config.IMAGE_WIDTH,  Config.IMAGE_HEIGHT, Config.Num_Classes,Config.classifier)
    EarlyStoppingMonitor            = EarlyStopping(patience=Config.epochs)

    #Encode labels to one hot vectors (ex : 2 -> [0,0,1,0,0,0,0,0,0,0])
    y_train_onehot                  = CNN.One_Hot_Encoding(y_train, Config.Num_Classes)
    y_test_onehot                   = CNN.One_Hot_Encoding(y_test , Config.Num_Classes)
    y_val_onehot                    = CNN.One_Hot_Encoding(y_val  , Config.Num_Classes)

    if Config.classifier =="softmax":

      model                       = Models.create_softmax_classifier(model, Config.Num_Classes)

      if Config.augment:

          train_generator, val_generator, test_generator = Data.Augment_x_train_flow(x_train, y_train_onehot, x_val, y_val,  x_test, y_test, Config.batch_size )
          history     = model.fit(train_generator, epochs= Config.epochs, validation_data=val_generator, callbacks= [EarlyStoppingMonitor])
          y_test      = np.argmax(y_test, axis=1)
          CNN.evaluate_generator(model, history , test_generator, y_test, Config.Num_Classes, Config.CMT_labels, Config.batch_size, Config.IMAGE_WIDTH)

      else:
          history     = model.fit(x = x_train, y = y_train_onehot,validation_data= (x_val, y_val_onehot),epochs = Config.epochs, batch_size = Config.batch_size,callbacks = [EarlyStoppingMonitor],shuffle= False)
          CNN.evaluate_x_test(model , history , x_test, y_test_onehot, Config.Num_Classes, Config.CMT_labels, Config.batch_size)

    else:

      Models.choose_classifier( x_train, y_train, x_test, y_test, Config.classifier,
                               Config.IMAGE_WIDTH,  Config.IMAGE_HEIGHT, Config.optimizeParam,
                               Config.optimizer, Config.CMT_labels, Config.Num_Classes)

    end2 = time.time()
    print("The duration of excuting this projct is :", (end2-start) /60, " Minutes")

    """ ******************************************************************************** """